In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import xarray as xr

In [2]:
%run create_JASADCP_metadata_df.ipynb

# Calculate Sv for each ncfile

### Narrow df down to Pacific region, OS instruments for now

In [16]:
print(df['geo_region'].isnull().sum())
print(len(df))
dfnow = df.dropna(subset=['geo_region'])
print(len(dfnow))
dfnow = dfnow[dfnow['geo_region'].str.contains('acific')]
print(len(dfnow))
dfnow = dfnow[dfnow['instrument_name'].str.contains('OS')]
print(len(dfnow))
dfnow = dfnow[(dfnow['instrument_name']!='OS-II-38')&(dfnow['instrument_name']!='OS-II-75')] 

## Load table of C, PDBW values

In [11]:
cpdbw = pd.read_csv(dpath + 'typical_system_characteristics_table2_mullison2017.csv')
cpdbw

## Load table of kc, Er values

## Load WOD data (T,S)

## Compute Sv

In [20]:
dfnow.iloc[0]

fname                                                       00804_short.nc
hardware_model                                           Ocean Surveyer 75
instrument_name                                                      OS-75
serial_numbers                                                        None
transmit_frequency                                                    None
phased_array                                                          None
cruise_beg_date                                        2004-06-15 06:33:10
blanking_interval                                                       8m
bin_length                                                              8m
transducer_beam_angle                                                 None
transmit_pulse_length                                                   8m
comments                 narrowband mode///mounted with the ADCP pointi...
biomass_dtmn                                                            no
geo_region               

In [ ]:
Sv = C + 

In [ ]:
ncfile = dpath + 'JASADCP/ncfiles/00001_short.nc'
nc = xr.open_dataset(ncfile)

In [ ]:
cols = df.columns.values

In [ ]:
cols[0:-1]

In [ ]:
nc

In [ ]:
nc['depth']

In [ ]:
nc['amp']

In [ ]:
nc['amp'].plot()

In [ ]:
nc['depth'].values

In [ ]:
#nct['ADCP_CONFIG'].attrs['model_name']
nct['ADCP_CONFIG'].attrs

# TESTING/OLD

In [ ]:
fnames = sorted(os.listdir(dpath + 'JASADCP/ncfiles'))

In [ ]:
nc_counter = len(fnames)
geo_region = [None]*nc_counter

In [ ]:
ifile = 0
for fname in fnames:
    ncfile = dpath + 'JASADCP/ncfiles/' + fname
    ncnow = xr.open_dataset(ncfile)
    strnow = ncnow.attrs['cruise_sonar_summary']
    geo_regionnow = re.findall("GEOGRAPHIC_REGION *: *((?:\S+ )*\S+)", strnow)
    if geo_regionnow:
        geo_region[ifile] = geo_regionnow[0]
    ifile = ifile+1

In [ ]:
df = pd.concat(
    [pd.Series(fnames,name='fname'), pd.Series(geo_region,name='geo_region')],
    axis=1)